# Power Plant Hedging Simulation

This notebook simulates a power plant’s operations and calculates a hedging strategy. The workflow includes:

1. Setting up the environment and simulation parameters.
2. Initializing forward curves and spread models.
3. Running the power plant simulation.
4. Executing the minimum variance hedging strategy.


---

In [11]:
import pandas as pd
import os

from forward_curve.forward_curve import generate_yearly_seasonal_curve, ForwardCurve
from hedging.min_var_hedge import MinVarHedge
from hedging.ops_plot_min_var_hedge import OpsPlotMinVarHedge
from market_simulation.spread_model.spread_model import SpreadModel
from pnl.hedge_pnl import HedgePnL
from valuation.operations_states import OperationalState
from valuation.power_plant.ops_plot_power_plant import OpsPlotPowerPlant
from valuation.power_plant.power_plant import PowerPlant
from base_config import CONFIG_FOLDER_ENV


# Set the environment variable for this notebook session
os.environ[CONFIG_FOLDER_ENV] = "C:\Projects\power-plant-rl\config"

## 1. Simulation Configuration
Set up the simulation period, number of Monte Carlo simulations, and paths for configuration files.

In [12]:
# Simulation configuration
config_path_spread_model = "model_configs/spread_model.json"
as_of_date = pd.Timestamp("2025-09-30")

simulation_start = as_of_date
simulation_end = pd.Timestamp("2026-12-31")
simulation_days = pd.date_range(start=simulation_start, end=simulation_end, freq="D")

n_sims = 1000

## 2. Initialize Forward Curves and Spread Model
We generate forward curves for power and coal and initialize the spread model.

In [13]:
# Generate initial forward curves
power_fwd_0 = generate_yearly_seasonal_curve(
    as_of_date=as_of_date,
    start_date=simulation_start,
    end_date=simulation_end,
    winter_value=130,
    summer_value=40.0,
)

coal_fwd_0 = ForwardCurve.generate_curve(
    as_of_date=as_of_date,
    start_date=simulation_start,
    end_date=simulation_end,
    start_value=90,
    end_value=80.0,
    name="Coal Forward Curve",
)

# Initialize spread model
spread_model = SpreadModel(as_of_date, simulation_days, config_path=config_path_spread_model)

# Simulate spot and forward prices
power_fwd, power_month_ahead, power_spot, coal_fwd, coal_month_ahead, coal_spot = spread_model.simulate(
    power_fwd_0=power_fwd_0, coal_fwd_0=coal_fwd_0, n_sims=n_sims, use_cache=True
)

[CACHE] Loaded cached simulation from C:\Users\danie\AppData\Local\Temp\2c8604723a95bc9eca426696b6c012e3
[CACHE] Loaded cached simulation from C:\Users\danie\AppData\Local\Temp\aac566730ea0c44876cebbe7b9e64708


## 3. Initialize and Optimize the Power Plant
Set up the PowerPlant object and run its optimization.

In [14]:
# Power plant simulation period
asset_start = pd.Timestamp(2025, 10, 1)
asset_end = asset_start + pd.Timedelta(days=364)
asset_days = pd.date_range(start=asset_start, end=asset_end, freq="D")

# Initial operational state
initial_state = OperationalState.IDLE

# Initialize PowerPlant
power_plant = PowerPlant(
    n_sims=n_sims,
    asset_days=asset_days,
    initial_state=initial_state,
    spots_power=power_spot,
    spots_coal=coal_spot,
    fwds_power=power_fwd,
    fwds_coal=coal_fwd,
    fwd_0_power=power_fwd_0,
    fwd_0_coal=coal_fwd_0,
    config_path="asset_configs/power_plant_config.yaml",
)

# Optimize plant operations
power_plant.optimize()

Optimizing simulation days: 100%|██████████| 364/364 [01:22<00:00,  4.42day/s]


## 4. Minimum Variance Hedging
Compute the hedge strategy for the power plant and roll out the cashflows of the hedge.

In [15]:
# Initialize hedging strategy
min_var_hedge = MinVarHedge(
    n_sims=n_sims,
    simulation_days=simulation_days,
    power_plant=power_plant,
    spots_power=power_spot,
    spots_coal=coal_spot,
    fwds_power=power_fwd,
    fwds_coal=coal_fwd,
)

# Execute hedge and calculate cashflows
min_var_hedge.hedge()
min_var_hedge.roll_out_cashflows()

# Plot the hedge effectiveness and R2
ops_plot_min_var_hedge = OpsPlotMinVarHedge(min_var_hedge)
ops_plot_min_var_hedge.plot_r2()
ops_plot_min_var_hedge.plot_hedge_effectiveness(power_plant.cashflows)

Determine minimal variance hedge: 365day [01:39,  3.66day/s]                        
C:\Projects\power-plant-rl\src\hedging\ops_plot_min_var_hedge.py:55: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
